# Setup External Access Integration for Open Meteo
This setup only needs to be done one.  I've done it for all students. You should have permission to enable use of this EAI in your notebook service.

To update your notebook service, look to the top of the page next to the blue "Run all" button. 
* Click the dropdown next to "Connected"
* Mouse down over the notebook service after it appears
* Click the "Edit" (pencil) icon that shows up when you mouse over the service
* Under "External Access Integrations" choose the "OPEN_METEO_INTEGRATION"

```sql
USE SCHEMA DATA5035.SPRING26;

CREATE NETWORK RULE open_meteo_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  SET VALUE_LIST = ('open-meteo.com','archive-api.open-meteo.com');

CREATE EXTERNAL ACCESS INTEGRATION open_meteo_integration
  ALLOWED_NETWORK_RULES = (open_meteo_rule)
  ENABLED = true;
  
GRANT USAGE ON NETWORK RULE open_meteo_rule TO ROLE snowflake_learning_role;

GRANT USAGE ON INTEGRATION open_meteo_integration TO ROLE snowflake_learning_role;

```

# Lab 04.1: Weather Data Analysis

This notebook fetches weather data from Open-Meteo for St. Louis, MO for January 2026 and counts the number of days with temperatures below freezing.

In [ ]:
import requests
import pandas as pd

# Q: How do we know the lat/long for St. Louis?
# A: You can look it up manually, or use a geocoding service like:
#    - Nominatim (OpenStreetMap): https://nominatim.openstreetmap.org/search?q=St.+Louis,+MO&format=json
#    - Google Geocoding API: https://developers.google.com/maps/documentation/geocoding
#    - geopy Python library: from geopy.geocoders import Nominatim; Nominatim().geocode("St. Louis, MO")
lat = 38.63
lon = -90.20
start_date = "2026-01-01"
end_date = "2026-01-31"

# Q: How do we know the format of the Open-Meteo URL?
# A: Open-Meteo provides interactive API documentation at:
#    - https://open-meteo.com/en/docs (current weather/forecast)
#    - https://open-meteo.com/en/docs/historical-weather-api (archive data)
#    The docs let you build queries interactively and show the resulting URL.
base_url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date,
    "end_date": end_date,
    "daily": "temperature_2m_min",
    "temperature_unit": "fahrenheit",
    "timezone": "America/Chicago"
}

# We use the Python requests package to do basic REST API interactions (get/post)
response = requests.get(base_url, params=params)
data = response.json()
data

In [ ]:
# The response is in JSON with each data series as a separate array,
# so we can easily extract just the elements we want.
df = pd.DataFrame({
    'date': data['daily']['time'],
    'min_temp_f': data['daily']['temperature_2m_min']
})
df

In [ ]:
# Now we'll filter the df to just those days below 32°F and count them.
freezing_days = df[df['min_temp_f'] < 32]
count_freezing = len(freezing_days)
print(f"Number of days with temperature below freezing (32°F): {count_freezing}")

In [ ]:
print(f"It reached below freezing on {len(freezing_days)*100 / len(df):0.0f}% of days in January!")